In [1]:
#default_exp source_to_refined

In [1]:
#export

from time import sleep
from nifi_api.environment import NifiIds
from nifi_api.dataflow import DataFlow

In [3]:
# export
def source_to_refined():
    """ The Source To Refined dataflow is decomposed in three sequential steps:
        - Source To Raw
        - Raw To Discovery
        - Discovery To Refined
    This function assures that each of these steps starts only after the
    previous step has finished.
    """

    # time between steps
    T = 15

    source_to_raw = DataFlow(NifiIds.sourceToRaw)
    raw_to_discovery = DataFlow(NifiIds.rawToDiscovery)
    discovery_to_refined = DataFlow(NifiIds.discoveryToRefined)

    source_to_raw.run()
    sleep(T)
    raw_to_discovery.run()
    sleep(T)
    discovery_to_refined.run()

In [4]:
source_to_refined()

Pipeline watching has finished ...
Pipeline watching has finished ...
Pipeline watching has finished ...
